# Onderzoek Openstreetmap wijken en buurten

In [12]:
# openstreetmap API package
# can be installed in conda cmd with:
# conda config --prepend channels conda-forge
# conda create -n ox --strict-channel-priority osmnx
from osmnx import geometries

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## pak de gezondheid wijken en buurten data

In [19]:
gezondheids_df = pd.read_csv('data/Gezondheid_per_wijk_en_buurt_2020.csv')
display(gezondheids_df)

,Leeftijd,Marges,Perioden,WijkenEnBuurten,Gemeentenaam,SoortRegio,Regioaanduiding/Codering (),ErvarenGezondheidGoed,Ondergewicht,NormaalGewicht,Overgewicht,ErnstigOvergewicht,VoldoetAanAlcoholRichtlijn,Drinker,ZwareDrinker,OvermatigeDrinker
0,18 jaar of ouder,Waarde,2020.0,Nederland,Nederland,Land,NL01,78.4,2.0,47.4,50.8,15.6,43.9,78.5,8.4,6.3
1,18 jaar of ouder,Waarde,2020.0,Aa en Hunze,Aa en Hunze,Gemeente,GM1680,80.5,1.7,45.0,53.9,15.8,42.9,80.1,7.2,6.6
2,18 jaar of ouder,Waarde,2020.0,Wijk 00 Annen,Aa en Hunze,Wijk,WK168000,81.4,1.6,46.7,53.0,15.2,40.1,80.9,7.0,5.9
3,18 jaar of ouder,Waarde,2020.0,Annen,Aa en Hunze,Buurt,BU16800000,81.3,1.6,46.4,53.2,15.3,40.0,80.7,7.0,5.8
4,18 jaar of ouder,Waarde,2020.0,Verspreide huizen Annen,Aa en Hunze,Buurt,BU16800009,84.4,1.6,51.9,49.1,12.4,42.2,85.0,7.2,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,18 jaar of ouder,Waarde,2020.0,Gerenbroek,Zwolle,Buurt,BU01935150,83.1,1.7,46.8,52.7,15.7,42.0,81.8,7.9,6.1
17337,18 jaar of ouder,Waarde,2020.0,Oude Mars,Zwolle,Buurt,BU01935160,92.3,1.7,59.0,40.7,9.8,33.5,91.0,9.2,6.4
17338,18 jaar of ouder,Waarde,2020.0,Wijk 52 Soestweteringlanden,Zwolle,Wijk,WK019352,84.1,1.8,49.2,49.8,13.0,38.7,83.4,9.7,7.3
17339,18 jaar of ouder,Waarde,2020.0,Windesheim,Zwolle,Buurt,BU01935200,84.3,1.7,48.1,50.9,13.8,37.7,83.2,10.6,7.8


### data opschonen

In [20]:
# Sum van alle Nan values per coloumn
print(f'Voordat we de null-waardes droppen hebben we: {len(gezondheids_df)}  in de dataset')
print(gezondheids_df.isnull().sum())
gezondheids_df.dropna(inplace=True)
# veel wijken en buurten bevatten als null waarde bij de gezondheidsdata een '.'
gezondheids_df = gezondheids_df.drop(gezondheids_df[gezondheids_df.Ondergewicht == '.'].index)
print(f'Nadat we de null-waardes droppen hebben we: {len(gezondheids_df)} in de  dataset')

Voordat we de null-waardes droppen hebben we: 17341  in de dataset
Leeftijd                       0
Marges                         5
Perioden                       5
WijkenEnBuurten                5
Gemeentenaam                   5
SoortRegio                     5
Regioaanduiding/Codering ()    5
ErvarenGezondheidGoed          5
Ondergewicht                   5
NormaalGewicht                 5
Overgewicht                    5
ErnstigOvergewicht             5
VoldoetAanAlcoholRichtlijn     5
Drinker                        5
ZwareDrinker                   5
OvermatigeDrinker              5
dtype: int64
Nadat we de null-waardes droppen hebben we: 16568 in de  dataset


## pak Openstreetmap data voor alcoholgebruik en obesitas.

Maak een lijst met alle query's voor het zoeken naar wijken en buurten.

In [28]:
places = []

for ind_r, row_b in gezondheids_df.iterrows():
    if str(row_b['SoortRegio']) in ['Gemeente', 'Land']:
        places.append([str(row_b['Gemeentenaam']) + ', Nederland', row_b['SoortRegio']])
    else:
        places.append([str(row_b['WijkenEnBuurten']) + ', ' + str(row_b['Gemeentenaam']) + ', Nederland', row_b['SoortRegio']])

print(places[100])

['Verspreide huizen Het Zwanenbroek, Aalten, Nederland', 'Buurt']


Kies welke facalliteiten gezocht moeten worden.

In [16]:
alcoholgebruik_tags = {'amenity': ['pub', 'bar']}
obesitas_tags = {'amenity': ['fast_food']}

zoek naar alle wijken en buurten voor de gekozen tags.

In [35]:
# lijsten om de data in op te slaan.
bar_data = []
fastfood_data = []
# var om het aantal errors te tellen van niet gevonden plaatsen
errors = 0

# loop door de wijken en buurten heen en zoek naar de voorzieningen.
print('start searching')
for place in places:
    
    # zoek in gemeente en land
    if place[1] in ['Gemeente', 'Land']:
        try:
            # zoek naar bars in de grenzen
            bar = geometries.geometries_from_place(place[0], alcoholgebruik_tags)
    
            # zoek naar fastfood restaurants in de grenzen
            fastfood = geometries.geometries_from_place(place[0], obesitas_tags)
            
        except ValueError:
            bar_data.append(None)
            fastfood_data.append(None)
            errors += 1
            continue

    # zoek in wijken en buurten
    else:
        try:
            # zoek naar bars in een afstand vanaf de wijk of buurt
            bar = geometries.geometries_from_address(place[0], alcoholgebruik_tags)
        
            # zoek naar fastfood restaurants in een afstand vanaf de wijk of buurt
            fastfood = geometries.geometries_from_address(place[0], obesitas_tags)
            
        except ValueError:
            bar_data.append(None)
            fastfood_data.append(None)
            errors += 1
            continue
    
    bar_data.append(len(bar))
    fastfood_data.append(len(fastfood))
    
    per = places.index(place) / len(places) * 100
    print('%d procent gezocht ;' % per + 'errors: ' + str(errors), end='\r')


start searching


KeyboardInterrupt: 